# Grade prediction with BERT embedding

In this notebook, we will use pre-trained deep learning model to process some text. We will then use the output of that model to classify the reviews we scrapped in previous courses. We will try to predict wether it was positive or negative. 

Under the hood, the model is actually made up of two model.


1.   DistilBERT processes the sentence and passes along some information it extracted from it on to the next model. DistilBERT is a smaller version of BERT developed and open sourced by the team at HuggingFace. It’s a lighter and faster version of BERT that roughly matches its performance.
2.   The next model, a basic Logistic Regression model from scikit learn will identify if the review was positive or negative



## Installing the transformers library

Let's start by installing the huggingface transformers library so we can load our deep learning NLP model.

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 11.6 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

## Importing the dataset

We'll import the dataset as we did in previous notebooks:

In [4]:
from google.colab import drive
import os
drive.mount('/content/drive')
dirpath = "drive" 
os.listdir(dirpath)

Mounted at /content/drive


['MyDrive', '.shortcut-targets-by-id', '.file-revisions-by-id', '.Trash-0']

In [ ]:
import os


# IMPORT YOUR REVIEWS FILE





In [ ]:
reviews.info()

In the first place we should determine what are we considering a positive review. In this case we shall consider as positive every comment that had 4 or 5 as grade and negative otherwise.

In [ ]:
# ASSIGN A LABEL TO EACH REVIEW TO DIFFERENCITATE POSITIVE AND NEGATIVE REVIEWS

# REMOVE REVIEWS WITH NULL VALUES


For computational purposes, we'll only use 1000 sentences which have less than 100 words in it.

In [ ]:
# Filter the reviews


In [ ]:
#Save in two variables the comments and the labels

sentences = 
labels = 

## Loading the Pre-trained BERT model
Let's now load a pre-trained BERT model.

In [ ]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Right now, the variable model holds a pretrained distilBERT model -- a version of BERT that is smaller, but much faster and requiring a lot less memory.

# Model 1: Preparing the Dataset
Before we can hand our sentences to BERT, we need to do some minimal processing to put them in the format it requires.

### Tokenization
Our first step is to tokenize the sentences -- break them up into word and subwords in the format BERT is comfortable with.

In [ ]:
# Apply tokenization

tokenized = 

### Padding
After tokenization, tokenized is a list of sentences -- each sentences is represented as a list of tokens. We want BERT to process our examples all at once (as one batch). It's just faster that way. For that reason, we need to pad all lists to the same size, so we can represent the input as one 2-d array, rather than a list of lists (of different lengths).


In [ ]:
# Perform the padding

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = 

Our dataset is now in the padded variable, we can view its dimensions below:

In [ ]:
np.array(padded).shape

### Masking
If we directly send padded to BERT, that would slightly confuse it. We need to create another variable to tell it to ignore (mask) the padding we've added when it's processing its input. That's what attention_mask is:

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

## Model 1: And Now, Deep Learning!
Now that we have our model and inputs ready, let's run our model! 

The model() function runs our sentences through BERT. The results of the processing will be returned into last_hidden_states.

In [ ]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

Let's slice only the part of the output that we need. That is the output corresponding the first token of each sentence. The way BERT does sentence classification, is that it adds a token called [CLS] (for classification) at the beginning of every sentence. The output corresponding to that token can be thought of as an embedding for the entire sentence.

We'll save those in the features variable, as they'll serve as the features to our logitics regression model.



In [ ]:
features = last_hidden_states[0][:,0,:].numpy()

## Model 2: Train/Test Split
Let's now split our datset into a training set and testing set (even though we're using 1,000 sentences from the reviews training set).

In [ ]:
# Split the data in train and test with the train_test_split function



## [Bonus] Grid Search for Parameters
We can dive into Logistic regression directly with the Scikit Learn default parameters, but sometimes it's worth searching for the best value of the C parameter, which determines regularization strength.

We now train the LogisticRegression model. If you've chosen to do the gridsearch, you can plug the value of C into the model declaration (e.g. LogisticRegression(C=5.2)).

In [ ]:
# Fit the logistic regression model

## Evaluating Model 2
So how well does our model do in classifying sentences? One way is to check the accuracy against the testing dataset:

In [ ]:
# Evaluate the score of the model

How good is this score? What can we compare it against? Let's first look at a dummy classifier:

In [ ]:

from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

# COmpare your model with a dummy classifier

So our model clearly does better than a dummy classifier